In [ ]:
import math
import os
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import TensorDataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import json
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OneHotEncoder

if torch.cuda.is_available():
  torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Hyperparameters

In [ ]:
# Transformer HyperHPeters
# The length of our token sequences
ntokens_HP = 30
# Number of attention heads
nheads_HP = 16
# The encoding dimensions of our tokens
token_encode_size_HP = 16
# The embedding string size
# Pytorch will cut this embedded sequence 
# and give an equal amount to each head, different 
# than in theory 
embed_size_HP = token_encode_size_HP *  nheads_HP
# Output size of the heads, which 
# learn an embedding.
head_embedsize_HP = 4
# No dropout for now
dropout_HP = 0.1
# Number of encoding layers
n_encoders_HP = 6

# Standard stuff
activation_HP = "relu"
layer_norm_eps_HP = 1e-5
batch_first_HP = True
norm_first_HP = False

# Trainig and validation Hyperparameters
datasetsize_HP = 1000000
split_HP = 0.9
batchsize_HP = 256
batchsize_HP = min(batchsize_HP, (int)(datasetsize_HP*0.2))
nepochs_HP = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_HP = nn.MSELoss()
learnrate_HP = 1e-4

# Import and preprocess the data

In [ ]:
# Json -> Dataframe
features_df = pd.read_json('strandenergylist.json')
energy_df = features_df[1]
energy_df = energy_df.head(datasetsize_HP)
features_df = features_df.head(datasetsize_HP)
features_df = features_df[0].str.split('', expand=True)
features_df = features_df.drop([0, 31], axis=1)
labels_df = features_df[1]


# Dataframe -> Tensors
dna = features_df.to_numpy()
for i in range(dna.shape[0]):
    for j in range(dna.shape[1]):
        if (dna[i,j] == 'A'): dna[i,j] = 0
        if (dna[i,j] == 'C'): dna[i,j] = 1
        if (dna[i,j] == 'G'): dna[i,j] = 2
        if (dna[i,j] == 'T'): dna[i,j] = 3
dna = dna.astype(int)
dna = torch.from_numpy(dna)
energy = torch.tensor(energy_df.values).to(torch.float)


# Tensors -> Dataloaders
dataset = TensorDataset(dna, energy)
ntrain = (int) (split_HP * datasetsize_HP)
ntest = datasetsize_HP - ntrain
train_set, vali_set = torch.utils.data.random_split(dataset, [ntrain, ntest], 
                                                    generator=torch.Generator(device='cuda'))

train_dataloader = DataLoader(train_set, batch_size=batchsize_HP, shuffle=True, 
                              generator=torch.Generator(device='cuda'))
vali_dataloader = DataLoader(vali_set, batch_size=1, shuffle=False, 
                             generator=torch.Generator(device='cuda'))

# Create the transformer model

In [ ]:
# Created entirely by the Pytorch team and pasted here.
# Adds information of position in the encoding of the tokens.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class RegressionTranformer(nn.Module):
    def __init__(self):
        super(RegressionTranformer, self).__init__()

        # Takes indexed sequences of tokens and embeds them.
        # [batch_size, seq_len] -> [seq_len, batch_size, embedding_dim] TODO check this!
        self.embedder = nn.Embedding(ntokens_HP, embed_size_HP)

        # Will add information about position in sequence to each token.
        # The second argument is the dropout probability.
        # [seq_len, batch_size, embedding_dim] -> [seq_len, batch_size, embedding_dim] TODO check this!
        self.posi_encoder = PositionalEncoding(embed_size_HP, 0.5)

        # The core of our regression transformer
        transformer_encoder_layer = nn.TransformerEncoderLayer(
            d_model = embed_size_HP,
            nhead = nheads_HP,
            dropout = dropout_HP,
            activation = activation_HP,
            layer_norm_eps = layer_norm_eps_HP,
            batch_first = batch_first_HP,
            norm_first = norm_first_HP
        )
        self.encoding_layers = nn.TransformerEncoder(transformer_encoder_layer, n_encoders_HP)

      
        # A linear map returning a scalar value, which is the predicted free
        # energy of the input sequence.
        # [batch_size, flatten_seq_len] -> [1]
        self.linear = nn.Linear(embed_size_HP * ntokens_HP, 1) 


    def forward(self, x):
        ŷ =  self.embedder(x)

        # Normalise the encoding.
        ŷ = ŷ * (1 / math.sqrt(embed_size_HP))

        # Add positionnal encoding
        ŷ = torch.permute(ŷ, (1,0,2)) # Necessary for the posi. enc. function.
        ŷ = self.posi_encoder(ŷ)

        # Apply the attention mechanisms
        ŷ = torch.permute(ŷ, (1,0,2)) # Back to normal
        ŷ = self.encoding_layers(ŷ)

        # Flatten the outputs
        ŷ = ŷ.flatten(1,2)

        # Apply last linear layer for regression
        ŷ = self.linear(ŷ)

        return ŷ
        
transformer = RegressionTranformer().to(device)

# Train the Transformer

In [ ]:
# Train # 1
def get_reg_accuracy():
    total = 0
    
    with torch.inference_mode():
      transformer.eval()
      for x_batch, y_batch in vali_dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            ŷ_batch = transformer(x_batch)
            batch_loss = loss_HP(ŷ_batch, y_batch.unsqueeze(1))
            total += batchsize_HP * batch_loss.item()

    transformer.train()
    return total / (datasetsize_HP*(1-split_HP))


transformer_optimizer = torch.optim.Adam(transformer.parameters(), lr=learnrate_HP)

trainloss = []
testloss = []

def trainroutine():
    global trainloss, testloss
    for epoch in range(nepochs_HP):
        running_loss = 0.0
        for i, (x_batch, y_batch) in enumerate(train_dataloader):
            # Put the data on the device
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Forward propagation
            ŷ_batch = transformer(x_batch)
            
            # Backpropagation
            batch_loss = loss_HP(ŷ_batch, y_batch.unsqueeze(1))
            transformer_optimizer.zero_grad()
            batch_loss.backward()
            transformer_optimizer.step()
            

            # Print statistics
            running_loss += batch_loss.item()*batchsize_HP
        
        # Print the results through the epochs
        trainloss = trainloss + [ round(running_loss/(datasetsize_HP*split_HP), 5) ]
        print("Training loss:", trainloss)
        testloss = testloss + [ round(get_reg_accuracy(), 5) ]
        print("Test loss:", testloss)
        
trainroutine()
torch.save(transformer.state_dict(),"transformer_weights.pt")


Training loss: [1.40719]
Test loss: [1.27038]
Training loss: [1.40719, 1.19603]
Test loss: [1.27038, 1.09003]
Training loss: [1.40719, 1.19603, 1.07546]
Test loss: [1.27038, 1.09003, 1.063]
Training loss: [1.40719, 1.19603, 1.07546, 1.03435]
Test loss: [1.27038, 1.09003, 1.063, 1.02674]
Training loss: [1.40719, 1.19603, 1.07546, 1.03435, 0.99871]
Test loss: [1.27038, 1.09003, 1.063, 1.02674, 0.9411]
Training loss: [1.40719, 1.19603, 1.07546, 1.03435, 0.99871, 0.92485]
Test loss: [1.27038, 1.09003, 1.063, 1.02674, 0.9411, 0.81243]
Training loss: [1.40719, 1.19603, 1.07546, 1.03435, 0.99871, 0.92485, 0.82452]
Test loss: [1.27038, 1.09003, 1.063, 1.02674, 0.9411, 0.81243, 0.72119]
Training loss: [1.40719, 1.19603, 1.07546, 1.03435, 0.99871, 0.92485, 0.82452, 0.55667]
Test loss: [1.27038, 1.09003, 1.063, 1.02674, 0.9411, 0.81243, 0.72119, 0.37143]
Training loss: [1.40719, 1.19603, 1.07546, 1.03435, 0.99871, 0.92485, 0.82452, 0.55667, 0.32237]
Test loss: [1.27038, 1.09003, 1.063, 1.02674, 0

In [ ]:
# Train # 2
def get_reg_accuracy():
    total = 0
    
    with torch.inference_mode():
      transformer.eval()
      for x_batch, y_batch in vali_dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            ŷ_batch = transformer(x_batch)
            batch_loss = loss_HP(ŷ_batch, y_batch.unsqueeze(1))
            total += batchsize_HP * batch_loss.item()

    transformer.train()
    return total / (datasetsize_HP*(1-split_HP))


transformer_optimizer = torch.optim.Adam(transformer.parameters(), lr=learnrate_HP)

trainloss = []
testloss = []

def trainroutine():
    global trainloss, testloss
    for epoch in range(nepochs_HP):
        running_loss = 0.0
        for i, (x_batch, y_batch) in enumerate(train_dataloader):
            # Put the data on the device
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Forward propagation
            ŷ_batch = transformer(x_batch)
            
            # Backpropagation
            batch_loss = loss_HP(ŷ_batch, y_batch.unsqueeze(1))
            transformer_optimizer.zero_grad()
            batch_loss.backward()
            transformer_optimizer.step()
            

            # Print statistics
            running_loss += batch_loss.item()*batchsize_HP
        
        # Print the results through the epochs
        trainloss = trainloss + [ round(running_loss/(datasetsize_HP*split_HP), 5) ]
        print("Training loss:", trainloss)
        testloss = testloss + [ round(get_reg_accuracy(), 5) ]
        print("Test loss:", testloss)
        
trainroutine()
torch.save(transformer.state_dict(),"transformer_weights.pt")

Training loss: [0.22026]
Test loss: [0.18343]
Training loss: [0.22026, 0.19513]
Test loss: [0.18343, 0.24517]
Training loss: [0.22026, 0.19513, 0.17647]
Test loss: [0.18343, 0.24517, 0.17352]
Training loss: [0.22026, 0.19513, 0.17647, 0.16067]
Test loss: [0.18343, 0.24517, 0.17352, 0.18647]
Training loss: [0.22026, 0.19513, 0.17647, 0.16067, 0.14853]
Test loss: [0.18343, 0.24517, 0.17352, 0.18647, 0.16439]
Training loss: [0.22026, 0.19513, 0.17647, 0.16067, 0.14853, 0.13872]
Test loss: [0.18343, 0.24517, 0.17352, 0.18647, 0.16439, 0.17732]
Training loss: [0.22026, 0.19513, 0.17647, 0.16067, 0.14853, 0.13872, 0.12975]
Test loss: [0.18343, 0.24517, 0.17352, 0.18647, 0.16439, 0.17732, 0.14472]
Training loss: [0.22026, 0.19513, 0.17647, 0.16067, 0.14853, 0.13872, 0.12975, 0.12222]
Test loss: [0.18343, 0.24517, 0.17352, 0.18647, 0.16439, 0.17732, 0.14472, 0.15336]
Training loss: [0.22026, 0.19513, 0.17647, 0.16067, 0.14853, 0.13872, 0.12975, 0.12222, 0.11627]
Test loss: [0.18343, 0.24517, 0

In [ ]:
# Train # 3
def get_reg_accuracy():
    total = 0
    
    with torch.inference_mode():
      transformer.eval()
      for x_batch, y_batch in vali_dataloader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            ŷ_batch = transformer(x_batch)
            batch_loss = loss_HP(ŷ_batch, y_batch.unsqueeze(1))
            total += batchsize_HP * batch_loss.item()

    transformer.train()
    return total / (datasetsize_HP*(1-split_HP))


transformer_optimizer = torch.optim.Adam(transformer.parameters(), lr=learnrate_HP)

trainloss = []
testloss = []

def trainroutine():
    global trainloss, testloss
    for epoch in range(nepochs_HP):
        running_loss = 0.0
        for i, (x_batch, y_batch) in enumerate(train_dataloader):
            # Put the data on the device
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            # Forward propagation
            ŷ_batch = transformer(x_batch)
            
            # Backpropagation
            batch_loss = loss_HP(ŷ_batch, y_batch.unsqueeze(1))
            transformer_optimizer.zero_grad()
            batch_loss.backward()
            transformer_optimizer.step()
            

            # Print statistics
            running_loss += batch_loss.item()*batchsize_HP
        
        # Print the results through the epochs
        trainloss = trainloss + [ round(running_loss/(datasetsize_HP*split_HP), 5) ]
        print("Training loss:", trainloss)
        testloss = testloss + [ round(get_reg_accuracy(), 5) ]
        print("Test loss:", testloss)
        
trainroutine()
torch.save(transformer.state_dict(),"transformer_weights.pt")

Training loss: [0.10611]
Test loss: [0.1198]
Training loss: [0.10611, 0.10136]
Test loss: [0.1198, 0.14323]
Training loss: [0.10611, 0.10136, 0.09722]
Test loss: [0.1198, 0.14323, 0.16825]
Training loss: [0.10611, 0.10136, 0.09722, 0.09356]
Test loss: [0.1198, 0.14323, 0.16825, 0.11015]
Training loss: [0.10611, 0.10136, 0.09722, 0.09356, 0.09133]
Test loss: [0.1198, 0.14323, 0.16825, 0.11015, 0.14036]
Training loss: [0.10611, 0.10136, 0.09722, 0.09356, 0.09133, 0.08628]
Test loss: [0.1198, 0.14323, 0.16825, 0.11015, 0.14036, 0.1212]
Training loss: [0.10611, 0.10136, 0.09722, 0.09356, 0.09133, 0.08628, 0.08416]
Test loss: [0.1198, 0.14323, 0.16825, 0.11015, 0.14036, 0.1212, 0.12244]
Training loss: [0.10611, 0.10136, 0.09722, 0.09356, 0.09133, 0.08628, 0.08416, 0.08096]
Test loss: [0.1198, 0.14323, 0.16825, 0.11015, 0.14036, 0.1212, 0.12244, 0.11991]
Training loss: [0.10611, 0.10136, 0.09722, 0.09356, 0.09133, 0.08628, 0.08416, 0.08096, 0.07822]
Test loss: [0.1198, 0.14323, 0.16825, 0.11

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, transformer.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

7905793
